In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import altair as alt
alt.data_transformers.enable('json')
alt.renderers.enable('jupyterlab')

RendererRegistry.enable('jupyterlab')

# User inputs

In [2]:
start_date = '20221130'
end_date = '20230509'
tidy_dataset_fn = f"tidy_df_{start_date}_{end_date}_noplanar_fit_clean.parquet"

# Load data

In [3]:
try:
    tidy_df = pd.read_parquet(
        tidy_dataset_fn
    )
except FileNotFoundError:
    print("No file such file exists for these dates.")
tidy_df['time'] = pd.to_datetime(tidy_df['time'])

In [4]:
# data = tidy_df.query("variable == 'T_3m_c'").set_index('time')['value']
# na_groups = data.notna().cumsum()[data.isna()]
# t_lengths_consecutive_na = na_groups.groupby(na_groups).agg(len)

# data = tidy_df.query("variable == 'RH_3m_c'").set_index('time')['value']
# na_groups = data.notna().cumsum()[data.isna()]
# rh_lengths_consecutive_na = na_groups.groupby(na_groups).agg(len)

# Examine pot. virtual temperature gradient

In [5]:
np.abs(tidy_df[tidy_df.measurement == 'temperature gradient'].value).min()

2.938309390776439e-06

In [6]:
alt.Chart(
    tidy_df.query("measurement == 'temperature gradient'").query("height < 5")
).mark_line().encode(
    x = 'time:T',
    y = 'value:Q',
    column='height:O'
).properties(width=200, height = 200) & alt.Chart(
    tidy_df.query("measurement == 'temperature gradient'").query("height < 5")
).mark_bar().encode(
    alt.X('value:Q').bin(step=0.1),
    alt.Y("count():Q"),    
    alt.Column('height:O')
).properties(width=200, height = 200)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [7]:
src = tidy_df.query("variable == 'temp_gradient_3m_c'")
neutral_times = src[src['value'].abs() < 0.01].time

In [8]:
alt.Chart(
    tidy_df[
        tidy_df.time.isin(neutral_times.sample(16))
    ].query("measurement == 'wind speed'").query("tower == 'c'")
).mark_line().encode(
    alt.X("value:Q").title("Wind speed (m/s)").sort('-y'),
    alt.Y("height:Q").title("Height (m)"),
    alt.Facet("time:O", columns=8)
).properties(width = 125, height = 125)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [9]:
z0_df = tidy_df[
    tidy_df.time.isin(neutral_times)
].query("tower == 'c'")
src_snowdepth = tidy_df[
    tidy_df.measurement == 'snow depth'
]
src_snowdepth = src_snowdepth[['time', 'value']].set_index('time').rename(columns={'value': 'snow_depth'})
z0_df = z0_df[z0_df.measurement.isin([
    'wind speed',
    'shear velocity',
    'snow depth'
])]
z0_df = z0_df[~z0_df.variable.str.contains("predicted")]
z0_df = z0_df.pivot_table(index=['time'], values='value', columns='variable')
z0_df = z0_df.join(src_snowdepth)
z0_df.head()

,SnowDepth_c,spd_10m_c,spd_15m_c,spd_20m_c,spd_2m_c,spd_3m_c,spd_5m_c,u*_10m_c,u*_15m_c,u*_20m_c,u*_2m_c,u*_3m_c,u*_5m_c,snow_depth
time,,,,,,,,,,,,,,
2022-12-01 18:00:00,0.069144,1.339750,1.315193,1.315227,1.239395,1.291831,1.318185,0.210201,0.206864,0.159162,0.119090,0.086317,0.140710,0.236545
2022-12-01 18:00:00,0.069144,1.339750,1.315193,1.315227,1.239395,1.291831,1.318185,0.210201,0.206864,0.159162,0.119090,0.086317,0.140710,0.069144
2022-12-02 15:00:00,0.100267,5.208190,5.368742,5.078640,3.821840,4.513449,5.341143,0.496962,0.251400,0.466722,0.219779,0.235802,0.319193,0.100267
2022-12-02 15:00:00,0.100267,5.208190,5.368742,5.078640,3.821840,4.513449,5.341143,0.496962,0.251400,0.466722,0.219779,0.235802,0.319193,0.374749
2022-12-02 17:30:00,0.100267,5.817068,5.610901,6.156200,5.685553,6.454514,6.070213,0.603661,0.834566,0.660502,0.270092,0.599743,0.402558,0.100267


# Filter out data without monotonically increasing wind speeds 

In [10]:
len(z0_df)

1130

In [11]:
def monotonically_increasing(l):
    return all(x < y for x, y in zip(l, l[1:]))

z0_df['is_monotonic_increasing'] = z0_df.apply(
    lambda row: monotonically_increasing([
            row['spd_2m_c'], 
            row['spd_3m_c'], 
            row['spd_5m_c'], 
            row['spd_10m_c'], 
            row['spd_15m_c'], 
            row['spd_20m_c']
    ]),
    axis = 1
)

In [12]:
z0_df = z0_df[z0_df.is_monotonic_increasing]

In [13]:
len(z0_df)

342

# Solve for $z_0$ assuming $d = 0$

https://www.eol.ucar.edu/content/calculation-roughness-length-and-displacement-height

In [14]:
von_karman = 0.4

In [15]:
d = 0.0
z0_df['z0_2m_c'] = (2 - d - z0_df['snow_depth'])/np.exp(z0_df['spd_2m_c']*von_karman/z0_df['u*_2m_c'])
z0_df['z0_3m_c'] = (3 - d - z0_df['snow_depth'])/np.exp(z0_df['spd_3m_c']*von_karman/z0_df['u*_3m_c'])
z0_df['z0_5m_c'] = (5 - d - z0_df['snow_depth'])/np.exp(z0_df['spd_5m_c']*von_karman/z0_df['u*_5m_c'])
z0_df['z0_10m_c'] = (10 - d - z0_df['snow_depth'])/np.exp(z0_df['spd_10m_c']*von_karman/z0_df['u*_10m_c'])
z0_df['z0_15m_c'] = (15 - d - z0_df['snow_depth'])/np.exp(z0_df['spd_15m_c']*von_karman/z0_df['u*_15m_c'])
z0_df['z0_20m_c'] = (20 - d - z0_df['snow_depth'])/np.exp(z0_df['spd_20m_c']*von_karman/z0_df['u*_20m_c'])

In [16]:
variables = [
        'z0_2m_c', 
        'z0_3m_c', 
        'z0_5m_c', 
        'z0_10m_c', 
        'z0_15m_c', 
        'z0_20m_c', 
    ]
alt.Chart(
    z0_df[variables].reset_index()
).transform_fold(
    variables
).transform_filter(
    alt.FieldOneOfPredicate('key', ['z0_2m_c', 'z0_3m_c'])
).mark_circle().encode(
    alt.X('time:T'),
    alt.Y("value:Q").scale(type='log'),
    alt.Row("key:N", sort=variables)
).properties(height = 100, width = 400)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [17]:
from sklearn.metrics import r2_score

In [18]:
alt.Chart(z0_df).mark_circle().encode(
    alt.X("z0_3m_c").scale(domain=[0.000000001, 10], type='log'),
    alt.Y("z0_2m_c").scale(domain=[0.000000001, 10], type='log')
).properties(width = 150, height = 150, title = str(round(r2_score(
        z0_df["z0_3m_c"],
        z0_df["z0_2m_c"]
    ), 3))
) | alt.Chart(z0_df).mark_circle().encode(
    alt.X("z0_3m_c").scale(domain=[0.000000001, 10], type='log'),
    alt.Y("z0_5m_c").scale(domain=[0.000000001, 10], type='log')
).properties(width = 150, height = 150, title = str(round(r2_score(
        z0_df["z0_3m_c"],
        z0_df["z0_5m_c"]
    ), 3))
) | alt.Chart(z0_df).mark_circle().encode(
    alt.X("z0_3m_c").scale(domain=[0.000000001, 10], type='log'),
    alt.Y("z0_10m_c").scale(domain=[0.000000001, 10], type='log')
).properties(width = 150, height = 150, title = str(round(r2_score(
        z0_df["z0_3m_c"],
        z0_df["z0_10m_c"]
    ), 3))
) | alt.Chart(z0_df).mark_circle().encode(
    alt.X("z0_3m_c").scale(domain=[0.000000001, 10], type='log'),
    alt.Y("z0_15m_c").scale(domain=[0.000000001, 10], type='log')
).properties(width = 150, height = 150, title = str(round(r2_score(
        z0_df["z0_3m_c"],
        z0_df["z0_15m_c"]
    ), 3))
) | alt.Chart(z0_df).mark_circle().encode(
    alt.X("z0_3m_c").scale(domain=[0.000000001, 10], type='log'),
    alt.Y("z0_20m_c").scale(domain=[0.000000001, 10], type='log')
).properties(width = 150, height = 150, title = str(round(r2_score(
        z0_df["z0_3m_c"],
        z0_df["z0_20m_c"]
    ), 3))
)

ValueError: Input contains NaN.

In [ ]:
alt.Chart(
    z0_df[['z0_3m_c']].resample("1D").median().reset_index()
).mark_circle(size=50).encode(
    alt.X("time:T"),
    alt.Y("z0_3m_c").scale(type='log')
) + alt.Chart(
    z0_df[['z0_3m_c']].resample("1D").median().reset_index()
).mark_bar(width=1).encode(
    alt.X("time:T"),
    alt.Y("z0_3m_c").scale(type='log')
).properties(height = 100)

In [ ]:
z0_df_weekly = z0_df[['z0_3m_c']].resample("W-MON").median().reset_index()

In [ ]:
z0_df[['z0_3m_c']].median()

In [ ]:
basic_z0_values_chart = alt.Chart(
    z0_df_weekly
).mark_circle(size=100).encode(
    alt.X("time:T"),
    alt.Y("z0_3m_c").scale(type='log')
) + alt.Chart(
    z0_df_weekly
).mark_bar(width=1).encode(
    alt.X("time:T"),
    alt.Y("z0_3m_c").scale(type='log')
).properties(height = 100)
basic_z0_values_chart

# Use Andreas et al. 2010 Method, NOAA/SPLASH (Chris Cox) solution

In [19]:
tdk = 273.15 
# surface temp mean
Tsm = tidy_df.query("variable == 'Tsurf_c'")['value'].values 
# air temp mean
Tam = tidy_df.query("variable == 'T_3m_c'")['value'].values 
# height of sonic
z_level_n = 3 - tidy_df.query("variable == 'SnowDepth_c'")['value'].values 
# wt-covariance, vertical flux of the sonic temperature  [deg m/s]
wT_csp = tidy_df.query("variable == 'w_tc__3m_c'")['value'].values   
wq_csp = tidy_df.query("variable == 'w_h2o__3m_c'")['value'].values   
wsp = tidy_df.query("variable == 'spd_3m_c'")['value'].values 
ustar = tidy_df.query("variable == 'u*_3m_c'")['value'].values 

surface_pot_temp = tidy_df.query("variable == 'Tsurfpot_c'")['value'].values
air_pot_temp  = tidy_df.query("variable == 'Tpot_3m_c'")['value'].values
surface_mixing_ratio = tidy_df.query("variable == 'Tsurfmixingratio_c'")['value'].values
air_mixing_ratio = tidy_df.query("variable == 'mixingratio_3m_c'")['value'].values
surface_specifichumidity = surface_mixing_ratio / (1 + surface_mixing_ratio)
air_specifichumidity  = air_mixing_ratio / (1 + air_mixing_ratio)

# Obukhov length
surflayr_avg_airtemp = 0.5*(Tsm + Tam)
surflayr_avg_specifichumidity = 0.5*(surface_specifichumidity + air_specifichumidity)
surflayr_avg_virtualtemp = 0.5*(
    tidy_df.query("variable == 'Tvirtual_3m_c'")['value'].values + 
    tidy_df.query("variable == 'Tsurfvirtual_c'")['value'].values
)
# right version of equation 2.3 in Andreas 2010
# L = - (
#     surflayr_avg_airtemp/( 0.4*9.81 )
# ) * (
#     ustar**3 / (
#         wT_csp + wq_csp* (0.61*surflayr_avg_airtemp)/(
#             1 + 0.61*surflayr_avg_specifichumidity
#           )
#     )
# )
# left version of equation 2.3 in Andreas 2010
L = - (
    (
        tidy_df.query("variable == 'Tvirtual_3m_c'")['value'].values
    )/( 0.4*9.81 )
) * (
    ustar**3 / wT_csp
)

# Monin-Obukhov stability parameter, z/L:
zeta_level_n = z_level_n/L

# Drag coefficient, Cd:
Cd = ustar**2/wsp**2 #- wu_csp/(wsp**2)

Ch = wT_csp / (wsp * (surface_pot_temp - air_pot_temp))

Ce = wq_csp / (wsp * (surface_specifichumidity - air_specifichumidity))

In [20]:
import sys
sys.path.append("../splash/")
import calc_z0
z0_values = calc_z0.calc_z0(z_level_n, Cd, zeta_level_n)
z0T_values = calc_z0.calc_z0T(z_level_n, Cd, Ch, zeta_level_n)
z0q_values = calc_z0.calc_z0Q(z_level_n, Cd, Ce, zeta_level_n)

time_values = tidy_df.time.unique()

z0_andreas_df = pd.DataFrame({
    "time": time_values, 
    "z0":   z0_values,
    "z0T": z0T_values,
    "z0q": z0q_values,
})

/Users/elischwat/Development/sublimationofsnow/analysis/paper1/../splash/calc_z0.py:16: RuntimeWarning: invalid value encountered in power
  sma = 1 + (6.5 * zL * (1+zL)**(1/3)) / (1.3 + zL); # Psi
/Users/elischwat/Development/sublimationofsnow/analysis/paper1/../splash/calc_z0.py:20: RuntimeWarning: invalid value encountered in power
  x = np.real((1 - 16*zL)**(0.25)) # assumes gamma = 16
/Users/elischwat/Development/sublimationofsnow/analysis/paper1/../splash/calc_z0.py:36: RuntimeWarning: invalid value encountered in power
  sma = 1 + (6.5 * zL * (1+zL)**(1/3)) / (1.3 + zL); # Psi
/Users/elischwat/Development/sublimationofsnow/analysis/paper1/../splash/calc_z0.py:37: RuntimeWarning: invalid value encountered in power
  x = np.real((1 - 16*zL)**(0.25)) # assumes gamma = 16
/Users/elischwat/Development/sublimationofsnow/analysis/paper1/../splash/calc_z0.py:44: RuntimeWarning: overflow encountered in exp
  z0 = z * np.exp(-(k*(Cd**0.5)*(Ch**-1) + sm))
/Users/elischwat/Development/subli

## Remove values >= 0.1, <= 7e-8 (Andreas et al., 2010)

In [21]:
print(len(z0_andreas_df.dropna()))

7073


In [22]:
z0_andreas_df['z0'] = z0_andreas_df['z0'].where(
    (z0_andreas_df['z0'] > 7e-8)
    &
    (z0_andreas_df['z0'] < 0.1)
)


In [23]:
print(len(z0_andreas_df.dropna()))

5255


## Apply the strict filtering criteria of Andreas et al. (2010)

In [26]:
time_values = tidy_df.query("variable == 'Tsurf_c'").set_index('time').sort_index().index.values
stress = tidy_df.query("variable == 'u*_3m_c'").assign(
    value = np.sqrt(tidy_df.query("variable == 'u*_3m_c'").value)
)
stress_good_times = stress[stress.value > 0].time

shflux = tidy_df.query("variable == 'w_tc__3m_c'")
shflux_good_times = shflux[np.abs(shflux.value) > 0.005].time

lhflux = tidy_df.query("variable == 'w_h2o__3m_c'")
lhflux_good_times = lhflux[np.abs(lhflux.value)/1000 > 2.5e-7].time

tdiff = (
    tidy_df[tidy_df.variable == 'Tsurfpot_c'].set_index('time')[['value']] - 
    tidy_df[tidy_df.variable == 'Tpot_3m_c'].set_index('time')[['value']]
)
tdiff_good_times = tdiff[np.abs(tdiff.value > 0.5)].index

Qdiff = (
    tidy_df[tidy_df.variable == 'Tsurfspecifichumidity_c'].set_index('time')[['value']] - 
    tidy_df[tidy_df.variable == 'specifichumidity_3m_c'].set_index('time')[['value']]
)
Qdiff_good_times = Qdiff[np.abs(Qdiff.value > 1e-5)].index

all_good_times = set(stress_good_times).intersection(
    set(shflux_good_times)
).intersection(
    set(lhflux_good_times)
).intersection(
    set(tdiff_good_times)
).intersection(
    set(Qdiff_good_times)
)

In [27]:
print(len(tidy_df.time.unique()))
print(len(stress_good_times))
print(len(shflux_good_times))
print(len(lhflux_good_times))
print(len(tdiff_good_times))
print(len(Qdiff_good_times))
print(len(all_good_times))

7728
7074
4404
6253
1895
5298
694


In [28]:
z0_andreas_df_strict = z0_andreas_df[z0_andreas_df.time.isin(all_good_times)]

In [42]:
nobs_times = tidy_df.query("variable == 'SF_avg_ue'").query("value == 0").time.values

In [43]:
z0_andreas_df_nobs = z0_andreas_df[z0_andreas_df.time.isin(nobs_times)]

In [29]:
print(len(z0_andreas_df.dropna()), len(z0_andreas_df_strict.dropna()))

5255 527


In [49]:
(
    (
        alt.Chart(
            np.log10(z0_andreas_df.set_index('time')).reset_index()
        ).mark_bar().encode(
            alt.X("z0:Q").bin(maxbins=30),
            alt.Y("count():Q")
        ) +
        alt.Chart(
            np.log10(z0_andreas_df.set_index('time')).reset_index()
        ).mark_rule(strokeWidth=3, strokeDash=[8,4], color='red').encode(
            alt.X("median(z0):Q")
        )
    ).properties(title=f"Median z0 = {round(z0_andreas_df.z0.median(), 6)}")
|
    (
        alt.Chart(
            np.log10(z0_andreas_df_strict.set_index('time')).reset_index()
        ).mark_bar().encode(
            alt.X("z0:Q").bin(maxbins=30),
            alt.Y("count():Q")
        ) +
        alt.Chart(
            np.log10(z0_andreas_df_strict.set_index('time')).reset_index()
        ).mark_rule(strokeWidth=3, strokeDash=[8,4], color='red').encode(
            alt.X("median(z0):Q")
        )
    ).properties(title=f"Median z0 = {round(z0_andreas_df_strict.z0.median(), 6)}")
|
    (
        alt.Chart(
            np.log10(z0_andreas_df_nobs.set_index('time')).reset_index()
        ).mark_bar().encode(
            alt.X("z0:Q").bin(maxbins=30),
            alt.Y("count():Q")
        ) +
        alt.Chart(
            np.log10(z0_andreas_df_nobs.set_index('time')).reset_index()
        ).mark_rule(strokeWidth=3, strokeDash=[8,4], color='red').encode(
            alt.X("median(z0):Q")
        )
    ).properties(title=f"Median z0 = {round(z0_andreas_df_nobs.z0.median(), 6)}")
).resolve_scale(x='shared')

/Users/elischwat/miniforge3/envs/sublimationofsnow/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: divide by zero encountered in log10
  result = func(self.values, **kwargs)
/Users/elischwat/miniforge3/envs/sublimationofsnow/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: divide by zero encountered in log10
  result = func(self.values, **kwargs)
/Users/elischwat/miniforge3/envs/sublimationofsnow/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: divide by zero encountered in log10
  result = func(self.values, **kwargs)
/Users/elischwat/miniforge3/envs/sublimationofsnow/lib/python3.12/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: divide by zero encountered in log10
  result = func(self.values, **kwargs)


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


## Calculate weekly medians

In [50]:
z0_andreas_df_weekly = z0_andreas_df.set_index('time').resample('W-MON').median().reset_index()
z0_andreas_df_strict_weekly = z0_andreas_df_strict.set_index('time').resample('W-MON').median().reset_index()
z0_andreas_df_nobs_weekly = z0_andreas_df_nobs.set_index('time').resample('W-MON').median().reset_index()

z0_andreas_df_weekly_counts = z0_andreas_df.set_index('time').resample('W-MON').count().reset_index()
z0_andreas_df_strict_weekly_counts = z0_andreas_df_strict.set_index('time').resample('W-MON').count().reset_index()
z0_andreas_df_nobs_weekly_counts = z0_andreas_df_nobs.set_index('time').resample('W-MON').count().reset_index()

In [53]:
z0_andreas_df_weekly = pd.merge(
    z0_andreas_df_weekly[['time', 'z0']].rename(columns={'z0': 'all data'}),
    z0_andreas_df_strict_weekly[['time', 'z0']].rename(columns={'z0': 'filtered'}),
    on='time',
    how='outer'
).merge(
    z0_andreas_df_nobs_weekly[['time', 'z0']].rename(columns={'z0': 'no bs'}),
    on='time',
    how='outer'
)

z0_andreas_df_weekly_counts = pd.merge(
    z0_andreas_df_weekly_counts[['time', 'z0']].rename(columns={'z0': 'all data'}),
    z0_andreas_df_strict_weekly_counts[['time', 'z0']].rename(columns={'z0': 'filtered'}),
    on='time',
    how='outer'
).merge(
    z0_andreas_df_nobs_weekly_counts[['time', 'z0']].rename(columns={'z0': 'no bs'}),
    on='time',
    how='outer'
).rename(columns = {
    'all data': 'all data counts',
    'filtered': 'filtered counts',
    'no bs': 'no bs counts'
})

z0_andreas_df_weekly = z0_andreas_df_weekly.merge(z0_andreas_df_weekly_counts, on='time')
z0_andreas_df_weekly

,time,all data,filtered,no bs,all data counts,filtered counts,no bs counts
0,2022-12-05,0.000327,0.001607,0.000282,226,12.0,176
1,2022-12-12,0.000137,0.003862,0.000080,247,5.0,194
2,2022-12-19,0.000282,0.000191,0.000091,304,22.0,212
3,2022-12-26,0.000115,0.000588,0.000080,253,30.0,188
4,2023-01-02,0.000044,0.000087,0.000044,225,4.0,152
5,2023-01-09,0.000154,0.000170,0.000161,210,55.0,172
6,2023-01-16,0.000054,0.000052,0.000037,212,28.0,174
7,2023-01-23,0.000085,0.000154,0.000057,253,18.0,197
8,2023-01-30,0.000086,0.000005,0.000079,290,19.0,198
9,2023-02-06,0.000043,0.000022,0.000045,250,35.0,200


In [55]:
print(z0_andreas_df.median())
print()
print(z0_andreas_df_strict.median())
print()
print(z0_andreas_df_nobs.median())

time    2023-02-18 11:45:00
z0                 0.000073
z0T                     0.0
z0q                0.112439
dtype: object

time    2023-02-22 23:45:00
z0                 0.000164
z0T               40.690011
z0q                0.423776
dtype: object

time    2023-02-17 02:45:00
z0                 0.000045
z0T                     0.0
z0q                0.063363
dtype: object


In [57]:
all_z0 = alt.Chart(
    z0_andreas_df
).mark_circle(opacity=0.25, size=5).encode(
    alt.X('time:T'),
    alt.Y("z0:Q").scale(type='log').axis(format="e"),
)
weekly_median_z0 = alt.Chart(
    z0_andreas_df_weekly.iloc[:-1]
).transform_fold(
    ['all data', 'filtered', 'no bs']
).mark_point(size=40, color='black').encode(
    alt.X('time:T'),
    alt.Y("value:Q").title("z₀"),
    alt.Shape("key:N")#.scale(range=['circle', 'triangle']).title(["Weekly", "average"])
)

median_line = alt.Chart(
    pd.DataFrame({'y':[z0_andreas_df.median().z0]})
).mark_rule(color='red', size=2).encode(y='y')
strict_median_line = alt.Chart(
    pd.DataFrame({'y':[z0_andreas_df_strict.median().z0]})
).mark_rule(color='red', size=2, strokeDash=[3,2]).encode(y='y')

z0_calculations_chart = (all_z0 + weekly_median_z0).properties(width = 250, height = 100)
z0_calculations_chart = (median_line + strict_median_line + z0_calculations_chart).configure_axis(grid=False)
z0_calculations_chart.save("z0_calculations_chart.png", ppi=400)
z0_calculations_chart.display(renderer='svg')

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


# Save roughness length values

In [59]:
z0_andreas_df.to_parquet("z0estimates/z0_andreas_df.parquet")
z0_andreas_df_strict.to_parquet("z0estimates/z0_andreas_df_strict.parquet")
z0_andreas_df_nobs.to_parquet("z0estimates/z0_andreas_df_nobs.parquet")
z0_andreas_df_weekly.to_parquet("z0estimates/z0_andreas_df_weekly.parquet")